### Imports inicales

In [3]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from IPython.display import display

#Libreria para plotear
# %pip install --upgrade plotly
import plotly.express as px

# Librerias utiles
# %pip install --upgrade scikit-learn
from sklearn.preprocessing import StandardScaler

#pip install fuzzywuzzy


In [4]:

def eda(dataframe):
    pd.options.display.max_rows = 200
    pd.set_option('display.max_columns', None)

    print('1.- El DataFrame tiene', dataframe.shape[0],'filas y', dataframe.shape[1], 'columnas\n')
    print('2.- El DataFrame está compuesto por las siguientes columnas: ', list(dataframe.columns),'\n')
    print('3.- Ejemplos de filas del DataFrame:\n')
    
    print('Primeras 5 filas:')
    display(dataframe.head(5))
    print('Últimas 5 filas:')
    display(dataframe.tail(5))
    print('Muestreo aleatorio de 5 filas:')
    display(dataframe.sample(5))
    print('4.- Descripción numérica del Dataframe:\n')
    display(dataframe.describe())
    print('5.- Cantidad de valores nulos por columna:')
    display(dataframe.isna().sum())
    print('6.- Cantidad de valores únicos por columna:')
    unique_values = dataframe.nunique()
    display(unique_values)
    print('7.- Filas totales duplicadas en el DataFrame:')
    display(dataframe.duplicated().sum())
    print('8.- Filas duplicadas en el DataFrame:')
    display(dataframe[dataframe.duplicated(keep=False)])

In [7]:
# restaurantes = pd.read_csv("./restaurants.csv")
# menus = pd.read_csv("./restaurant-menus.csv")

### Nexrt steps:
- Preprocesamiento: Limpiar la data, quitar columnas innecesarias y nulos. Cambiar precios a números
- Definir estados a trabajar
- Nueva columna de franquisia acsociada: Se contrulle dándole un diccionario y ver si el nombre calza con un nombre de cadena.
- Usar mismo csv para mabas cosas



Nos vamos aquedar ocn datos de ciertos estados

- Vsualización PowerBi:

 Mapa de algo, 
Comparar precio de la misma categorpia entre restaruantes vs sus rating
Puntación vs precio


Catgorías de restaruantes que más se repiten por estado
N° de franquisias por estado
Precio de la misma franquisia entre estados


-D3:
- Gráfico de barras pra comprar precios entre estados de la misma categoria
- Grafico de barras con cadenas por estado y al poner mouse encima aparace distribución de franquicias

La siguiete parte procesa los dos DF originales y deja solo los datos de cadenas de comida rápida relevantes que queremos usar:

In [2]:

# Esta parte quita las direcciones delos nombres de las cadenas

# Cargar el DataFrame 'restaurantes' (asumiendo que ya lo has cargado)
# restaurantes = pd.read_csv('ruta_a_tu_archivo.csv')

# Crear una copia del DataFrame para trabajar con ella
df_copia0 = restaurantes.copy()

# Función para eliminar texto entre paréntesis
def eliminar_direccion(nombre):
    # Elimina el texto entre paréntesis y los paréntesis mismos
    return re.sub(r'\(.*?\)', '', nombre).strip()

# Aplicar la función para limpiar los nombres
df_copia0['nombre_limpio'] = df_copia0['name'].apply(eliminar_direccion)

# Contar la frecuencia de cada nombre limpio
frecuencias = df_copia0['nombre_limpio'].value_counts()

# Filtrar los nombres que se repiten 3 veces o más
nombres_repetidos = frecuencias[frecuencias >= 3].index

# Quedarse solo con los nombres que se repiten 3 veces o más
df_sin_direccion = df_copia0[df_copia0['nombre_limpio'].isin(nombres_repetidos)]

# Mostrar las primeras filas del DataFrame final para verificar los resultados
df_sin_direccion



# Esto crea la columna nombre_caednas y le asigna el nombre de la cadena que esté en el top50, al resto lo deja en nulo

# Lista de cadenas
cadenas = [
    'Subway', 'Starbucks', 'McDonald’s', 'Dunkin’', 'Burger King', 'Taco Bell',
    'Pizza Hut', 'Domino’s', 'Wendy’s', 'Dairy Queen', 'Little Caesars', 'KFC',
    'Sonic Drive-In', 'Arby’s', 'Papa John’s', 'Chipotle', 'Jimmy John’s', 
    'Popeyes Louisiana Chicken', 'Chick-fil-A', 'Baskin-Robbins', 'Panda Express', 
    'Jack in the Box', 'Panera Bread', 'Jersey Mike’s', 'Hardee’s', 'Wingstop', 
    'Five Guys', 'Papa Murphy’s', 'Firehouse Subs', 'Carl’s Jr.', 'Marco’s Pizza', 
    'Tropical Smoothie Cafe', 'Church’s Chicken', 'Zaxby’s', 'Checkers and Rally’s', 
    'Whataburger', 'Culver’s', 'Bojangles', 'Qdoba', 'Moe’s', 'Tim Hortons', 
    'Del Taco', 'Steak ‘n Shake', 'Raising Cane’s', 'El Pollo Loco', 'McAlister’s Deli', 
    'Freddy’s Frozen Custard & Steakburgers', 'Krispy Kreme', 'White Castle', 
    'In-N-Out Burger'
]

# Normalización de cadenas
def normalizar(texto):
    texto = texto.lower()  # Convertir a minúsculas
    texto = re.sub(r'[^\w\s]', '', texto)  # Eliminar signos de puntuación
    texto = texto.replace(' ', '')  # Eliminar espacios
    return texto

# Normalizar la lista de cadenas
cadenas_normalizadas = {normalizar(cadena): cadena for cadena in cadenas}

# Crear una copia del DataFrame para trabajar con ella
df_copia1 = df_sin_direccion.copy()

# Seleccionar solo las columnas 'id', 'nombre_limpio' y 'name'
df_copia1 = df_copia1[['id', 'nombre_limpio', 'name']]

# Normalizar la columna 'nombre_limpio'
df_copia1['nombre_limpio_normalizado'] = df_copia1['nombre_limpio'].apply(normalizar)

# Función para asignar el nombre de la cadena basado en la normalización
def asignar_nombre_cadena(nombre_normalizado):
    if nombre_normalizado in cadenas_normalizadas:
        return cadenas_normalizadas[nombre_normalizado]
    return None

# Crear la nueva columna 'nombre_cadena' aplicando la función
df_copia1['nombre_cadena'] = df_copia1['nombre_limpio_normalizado'].apply(asignar_nombre_cadena)

# Eliminar la columna de normalización adicional si ya no es necesaria
df_copia1 = df_copia1.drop(columns=['nombre_limpio_normalizado'])

# Mostrar las primeras filas del DataFrame copiado para verificar los resultados
df_copia1



# Filtrar las filas donde 'nombre_cadena' es nulo
df_nulos = df_copia1[df_copia1['nombre_cadena'].isna()]

# Contar las apariciones de cada valor en 'nombre_limpio'
conteo_nombre_limpio = df_nulos['nombre_limpio'].value_counts()

# Filtrar para obtener solo los nombres que aparecen más de 100 veces
nombres_mas_de_100 = conteo_nombre_limpio[conteo_nombre_limpio > 100]

# Convertir a una lista si es necesario
nombres_mas_de_100_lista = nombres_mas_de_100.index.tolist()



# Crear un diccionario para mapear nombres frecuentes a ellos mismos (o nombres de cadenas si ya se sabe)
# En este caso, el nombre del mapeo es el mismo ya que estamos reemplazando por el mismo valor
df_copia2 = df_copia1.copy()

nombre_mapeo = {nombre: nombre for nombre in nombres_mas_de_100_lista}

# Actualizar 'nombre_cadena' en df_copia2 usando el diccionario para filas que coinciden con nombres_mas_de_100
df_copia2['nombre_cadena'] = df_copia2.apply(
    lambda row: nombre_mapeo.get(row['nombre_limpio'], row['nombre_cadena']),
    axis=1
)


# Lista de nombres de establecimientos que no son comida rápida
no_comida_rapida = [
    'The Ice Cream Shop', 'Cracker Barrel Old Country Store', 'Outback Steakhouse',
    'Walgreens', 'Bed Bath & Beyond', 'CVS', 'Safeway Express', 'ProFlowers', 'Rite Aid', '7-Eleven'
]

# Filtrar el DataFrame para eliminar las filas con esos nombres en 'nombre_cadena'
df_copia3 = df_copia2[~df_copia2['nombre_cadena'].isin(no_comida_rapida)]

# Realizar la fusión de los DataFrames basado en la columna 'id'
df_filtrado = pd.merge(restaurantes, df_copia3[['id', 'nombre_cadena']], on='id', how='inner')

# Filtrar las filas donde 'nombre_cadena' no es nulo
df_no_null = df_filtrado[df_filtrado['nombre_cadena'].notna()]

# Reordenar las columnas para que 'nombre_cadena' sea la cuarta columna
# Primero obtenemos el nombre de las columnas
cols = df_no_null.columns.tolist()

# Aseguramos que 'nombre_cadena' está en la lista de columnas
if 'nombre_cadena' in cols:
    # Reordenamos las columnas para que 'nombre_cadena' sea la cuarta
    cols.remove('nombre_cadena')
    cols.insert(3, 'nombre_cadena')  # Insertar 'nombre_cadena' en la cuarta posición
    df_no_null = df_no_null[cols]


# Exportar el DataFrame filtrado a un archivo CSV
# df_no_null.to_csv('restaurantes-cadenas.csv', index=False)

# Filtrar menus para conservar solo las filas con ids que están en df_no_null
df_filtrado_menus = menus[menus['restaurant_id'].isin(df_no_null['id'])]

#df_filtrado_menus.to_csv('menus-cadenas.csv', index=False)

NameError: name 'restaurantes' is not defined

### Ahora limpaimos la data

In [13]:
#cadenas = pd.read_csv("./restaurantes-cadenas.csv")
import pandas as pd
import csv

try:
    menuCadenas = pd.read_csv(
        "./menus-cadenas.csv",
        quoting=csv.QUOTE_NONE,       # Ajusta según sea necesario
        escapechar='\\',              # Escapa caracteres problemáticos
        on_bad_lines='skip'           # Omite líneas problemáticas
    )
except Exception as e:
    print(f"Error al leer el archivo: {e}")


In [14]:
menuCadenas

restaurant_id  \
4                          New! Crustless Pizzas - Keto Friendly (Baking R... Chicken Bacon Artichoke - Keto Friendly (Baking... "Creamy Garlic Sauce                                Whole-Milk Mozzarella                         Grilled Chicken Raised Without Antibiotics         Crispy Bacon     Marinated Artichoke Hearts  Fresh Spinach                     Aged Parmesan and Zesty Herbs without the Cru...   
                                                                              Cowboy - Keto Friendly (Baking Required)           "Traditional Red Sauce                              Whole-Milk Mozzarella                         Premium Pepperoni                                  Italian Sausage  Sliced Mushrooms            Black Olives                                                               Cheddar   
To make this Keto-friendly  simply change the Traditional Red Sauce to Cre...  Proteins: 75g                                      Carbs: 28g"                                       10.99 USD                                     NaN                                                NaN              NaN                         NaN                                                                             NaN   
4                          New! Crustless Pizzas - Keto Friendly (Baking R... Create Your Own Crustless - Keto Friendly (Baki... "Choose your sauce (or no sauce at all!) and ad... 9.0 USD                                       NaN                                                NaN              NaN                         NaN                                                                             NaN   
                                                                              Papa's All Meat - Keto Friendly (Baking Required)  "Traditional Red Sauce                              Whole-Milk Mozzarella                         Canadian Bacon                                     Salami           Premium Pepperoni           Italian Sausage and Ground Beef                                      and Cheddar.    
...                                                                                                                                                                                                                                                                                                                                                                                                               ...   
1833                       Beverages                                          Dr. Pepper                                         "Regular (240 Cal.)                                 Large (360 Cal.) Dr. Pepper. Please note      this beverage is self-serve when ordering In-C... 3.19 USD         NaN                         NaN                                                                             NaN   
                                                                              Mountain Dew                                       "Regular (280 Cal.)                                 Large (420 Cal.) Mountain Dew. Please note    this beverage is self-serve when ordering In-C... 3.19 USD         NaN                         NaN                                                                             NaN   
                                                                              Sierra Mist                                        "Regular (260 Cal.)                                 Large (380 Cal.) Sierra Mist. Please note     this beverage is self-serve when ordering In-C... 3.19 USD         NaN                         NaN                                                                             NaN   
                                                                              Bubly Lime                                         "Regular (0 Cal.)                                   Large (5 Cal.) Bubly Lime Allergens: none"   3.19 USD                                           NaN              NaN     

In [ ]:
cadenas = cadenas.dropna()
print(cadenas['price_range'].unique())

# Reemplazar puntos por comas en las columnas 'lat' y 'lng'
cadenas['lat'] = cadenas['lat'].astype(str).str.replace('.', ',', regex=False)
cadenas['lng'] = cadenas['lng'].astype(str).str.replace('.', ',', regex=False)

# Función para extraer el estado del formato de dirección
def extraer_estado(direccion):
    partes = direccion.split(',')
    if len(partes) >= 3:
        return partes[2].strip()
    return None

# Aplicar la función para crear una nueva columna 'estado'
df_cadenas['estado'] = df_cadenas['full_address'].apply(extraer_estado)




In [ ]:
# Función para limpiar y convertir el precio a numérico
def limpiar_precio(precio):
    try:
        # Eliminar " USD" y convertir a número
        return float(precio.replace(' USD', '').replace(',', '.'))
    except:
        return None

# Aplicar la función para transformar la columna 'price'
menuCadenas['price'] = menuCadenas['price'].apply(limpiar_precio)
